# Feature Engineering — Başlangıç

Bu bölümde, makine öğrenimi modeline girecek özellikleri (features) üretmeden önce veri setini
tekrar kolon bazında inceleyip, her tablonun hangi bilgiyi taşıdığını ve bu bilgilerin
feature engineering aşamasında nasıl değerlendirilebileceğini netleştirmeyi amaçlıyorum.

EDA sürecinde veriyi geniş açıdan analiz ettim ancak FE aşamasında daha detaylı ve
kolon-odaklı bir bakış gerekiyor. Bu nedenle:

- **orders**
- **order_products__prior**
- **order_products__train**
- **products**
- **aisles**
- **departments**

tablolarını yeniden, daha mikro seviyede inceleyeceğim.

Bu incelemelerin amacı:
- Hangi kolonların kullanıcı davranışını temsil ettiğini belirlemek  
- Hangi kolonların ürün özelliklerine sinyal taşıdığını anlamak  
- Kullanıcı–ürün etkileşimini modellemek için fırsatları fark etmek  
- Sonrasında üreteceğim yaratıcı FE’lerin temelini oluşturmak  



In [1]:
import warnings
warnings.filterwarnings("ignore")


import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

# Veri yollarını belirleme
DATA_RAW = Path("../data/raw")
DATA_INTERIM = Path("../data/interim")
FIGURES = Path("../figures")

# Gerekli klasörler oluşturma
DATA_INTERIM.mkdir(exist_ok=True, parents=True)

# Veriyi yükleme
orders = pd.read_csv(DATA_RAW / "orders.csv")
order_products_prior = pd.read_csv(DATA_RAW / "order_products__prior.csv")
order_products_train = pd.read_csv(DATA_RAW / "order_products__train.csv")
products = pd.read_csv(DATA_RAW / "products.csv")
aisles = pd.read_csv(DATA_RAW / "aisles.csv")
departments = pd.read_csv(DATA_RAW / "departments.csv")

#kontrol
print("Tüm tablolar yüklendi.")
print(f"orders: {orders.shape}")
print(f"order_products_prior: {order_products_prior.shape}")
print(f"order_products_train: {order_products_train.shape}")
print(f"products: {products.shape}")
print(f"aisles: {aisles.shape}")
print(f"departments: {departments.shape}")


Tüm tablolar yüklendi.
orders: (3421083, 7)
order_products_prior: (32434489, 4)
order_products_train: (1384617, 4)
products: (49688, 4)
aisles: (134, 2)
departments: (21, 2)


In [4]:
print("Orders Tablosu")
print(orders.head())
print("\nEksik Değerler:")
print(orders['days_since_prior_order'].isnull().sum())

print("\nProducts Tablosu")
print(products.head())
print("\nEksik Değerler:")
print(products.isnull().sum())

Orders Tablosu
   order_id  user_id eval_set  order_number  order_dow  order_hour_of_day  \
0   2539329        1    prior             1          2                  8   
1   2398795        1    prior             2          3                  7   
2    473747        1    prior             3          3                 12   
3   2254736        1    prior             4          4                  7   
4    431534        1    prior             5          4                 15   

   days_since_prior_order  
0                     NaN  
1                    15.0  
2                    21.0  
3                    29.0  
4                    28.0  

Eksik Değerler:
206209

Products Tablosu
   product_id                                       product_name  aisle_id  \
0           1                         Chocolate Sandwich Cookies        61   
1           2                                   All-Seasons Salt       104   
2           3               Robust Golden Unsweetened Oolong Tea        94   


In [5]:
print("\nAisles Tablosu")
print(aisles.head())
print("\nEksik Değerler:")
print(aisles.isnull().sum())

print("\nDepartments Tablosu")
print(departments.head())
print("\nEksik Değerler:")
print(departments.isnull().sum())


Aisles Tablosu
   aisle_id                       aisle
0         1       prepared soups salads
1         2           specialty cheeses
2         3         energy granola bars
3         4               instant foods
4         5  marinades meat preparation

Eksik Değerler:
aisle_id    0
aisle       0
dtype: int64

Departments Tablosu
   department_id department
0              1     frozen
1              2      other
2              3     bakery
3              4    produce
4              5    alcohol

Eksik Değerler:
department_id    0
department       0
dtype: int64


In [6]:
print("\nOrder Products Prior Tablosu")
print(order_products_prior.head())
print("\nEksik Değerler:")
print(order_products_prior.isnull().sum())

print("\nOrder Products Train Tablosu")
print(order_products_train.head())
print("\nEksik Değerler:")
print(order_products_train.isnull().sum())


Order Products Prior Tablosu
   order_id  product_id  add_to_cart_order  reordered
0         2       33120                  1          1
1         2       28985                  2          1
2         2        9327                  3          0
3         2       45918                  4          1
4         2       30035                  5          0

Eksik Değerler:
order_id             0
product_id           0
add_to_cart_order    0
reordered            0
dtype: int64

Order Products Train Tablosu
   order_id  product_id  add_to_cart_order  reordered
0         1       49302                  1          1
1         1       11109                  2          1
2         1       10246                  3          0
3         1       49683                  4          0
4         1       43633                  5          1

Eksik Değerler:
order_id             0
product_id           0
add_to_cart_order    0
reordered            0
dtype: int64


> `orders` tablosu:   

- order_id: uniqe kimlik   
- user_id: sipariş veren kullanıcıların uniqe kimliği 
 >FE: **toplam sipariş sayısı + ilk siparişten bu güne kaç gün geçtiği + son 30 günde kaç sipariş verdiği(sipariş sıklığına göre trendyol+amazon vb. gibi sitelerden gelen bildirimler doğrultusunda)** 
- eval_set: eda ve baseline sürecinde anladığım, her siparişin hangi amaçla kullanıldığını belirten kolon, 'prior' geçmiş siparişler, 'train' model için hedef verisi olacak kullanılacak  
- **order_number:kullanıcının kaçıncı siparişi olduğu,kullanıcı alışkanlıklarını anlamak için önemli.  FE süreci için önemli bir kolon** 
>FE: **toplam sipariş, sipariş sayısına göre yeni/eski kullanıcı olması** 
- **order_dow:  Siparişin verildiği gün. haftalık alışveriş davranışlarını gösteriyor. FE için önemli kolon**
>FE: **EDA sonucu en yoğun sipariş günleri: PAZAR-PAZARTESİ, kullanıcının haftanın hangi gün sipariş etiğiyle alaklı bir FE**      
- **order_hour_of_day: kullanıcıların sipariş zamanlarını gösteriyor, FE için önemli**
>FE: **Sipariş saatine göre EDA'dan anladığımız: Planlı alışveriş + ani ihtiyaç doğrultusunda alışveriş**    
- **days_since_prior_order: son siparişten itibaren geçen gün sayısı, FE için** 
>FE: **Son siparişten bu yana kaç gün geçtiği + En uzun sipariş aralığı (bununla 30. Günde aylık sipariş yapanlar ve haftalık market alışverişi yapanları anlayabilirim.)**  

> `Products`(ürün) tablosu:    

- product_id: ürün hakkında detaylı bilgiye sahip olmak için
>FE: **Kaç kez sipariş verdiği + ne sıklıkla reorder ettiği (yüksek(temel ihtiyaç), düşük(ani ihtiyaç))+kaç farklı kullanıcının satın aldığı+kullanıcının kaç defa aynı ürünü aldığı(yüksek FE)**    
- product_name: ürün ismi önemli gibi, en çok tüketilen ürünler alışveriş alışkanlıkları hk. bilgi vermişti 
>FE: **en çok satın alınan ürünler organic kategorisi olduğu için sağlıklı ürün oranı**  
- aisle_id: ürünleri kategorik olarak anlamayı sağlayacak   
>FE: **En çok hangi koridordan ürün aldığı + kaç farklı koridordan alışveriş yaptığı üzerine FE iyi olur**
- department_id: departman, product_name gibi kullanıcı davranışlarını anlamamı sağlayacak
>FE: **Favori departmanı+kaç farklı departmandan alışveriş yaptığı**   

> `Aisles`(koridor) tablosu:    

- **aisle_id: kategorilerin ve ürünler tablosunun bağlantı noktası.  FE için önemli kolan**   
- aisle: kategorileri daha anlamlı hale getirebilir   

> `Departments` tablosu:  

- department_id: ürünlerin hangi departmana ait olduğunu anlayacağım, Products' tablosu ile bağlantı için önemli  
- **department: EDA Sürecinde gördüğüm kullanıcıların tekrarlayan alışverişleri departmanlara sadık oluşu modellemede önemli sonuçlar çıakracak**  


> `Order_Products_Prior` tablosu:  

- **geldik o malum tabloya**
- order_id: orders tablosu ile bağlantı kurmaya yaracayacak , önceki siparişteki ürünleri tanımlayacak  
- **product_id: produc tablosu ile bağlantı kuracak, ürünü tekrar satın alıp almayacağını anlamak için kritik**  
>FE: product tablosunda detaylı anlatım.
- ##### add_to_cart_order: ürünnün sepete eklenme sırası, alışveriş sırasını anlamak için kıymetli, EDA sürecinde anladığımız ilk eklenen ürünler genelde en temel ihtiyaçlardır 
>FE: **ilk eklenenler temel alışveriş mantığıyla: ürünün kaçıncı sırayla sepete girdiği** 
- ##### reordered: ürün tekrar alındı mı sorusunun kaynağı.. modelin hedef değişkeni olarak kullanacağım. kulllanıcı alışkanlıkları, ürün popüleritesini anlamak için önemli.
>FE: **kullanıcının genel reorder oranı üzerinden sadık ya da yeni ürünler aldığını bulabiliriz + son 10 siparişte yeni ürün oranı ile bunu anlayabiliriz**

> `Order_Products_Train` tablosu:  

- **order_id: modelin eğitim verisi, gerçek hedef değişkenine sahip**   
- **product_id: ürünlerin tekrar alınma oranları modelin çıktısını etkileyecek**   
- ##### reordered: modelin eğitileceği hedef değişken. bu FE'den üretilen reorder_ratio" tek FE ile -F1: 0.7452 skoru verdi. EN ÖNEMLİ FE ÜRETİM BANTLARINDAN BİRİSİ OLACAK.**  

Orders tablosundan çıkaracağım FE’ler:

- user_total_orders : kullanıcının bugüne kadar verdiği toplam sipariş
- user_avg_days_between_orders : ortalama sipariş aralığı
- user_max_gap :  kullanıcının en uzun alışveriş aralığı
- user_weekly_ratio :  siparişlerinin haftalık döngüye (5–9 gün arası) uyma oranı
- user_monthly_ratio :  yaklaşık her 30 günde bir sipariş verme davranışı
- user_favorite_dow :  kullanıcının en çok sipariş verdiği gün
- user_favorite_hour :  sipariş verdiği en yoğun saat
- user_recent_gap : train setindeki son siparişten bu yana geçen gün
- user_reorder_ratio : kullanıcının “tekrar alma” eğilimi 

In [ ]:
vars_to_clear = ["prior_with_user", "user_total_orders", "user_weekend_ratio",
                 "user_favorite_dow", "user_peak_ratio", "user_night_ratio",
                 "user_avg_gap", "user_max_gap", "user_weekly_ratio",
                 "user_monthly_ratio", "user_reorder_ratio", "user_exploration",
                 "user_features"]

for v in vars_to_clear:
    if v in globals():
        del globals()[v]

print("Temiz başlangıç yapıldı.\n")


# 1) Prior siparişlere user_id + order metadata merge
prior_with_user = (
    order_products_prior
    .merge(
        orders[['order_id', 'user_id', 'order_number', 'days_since_prior_order',
                'order_dow', 'order_hour_of_day']],
        on='order_id',
        how='left'
    )
)

print("prior_with_user:", prior_with_user.shape)


# 2) Kullanıcı sipariş davranışı
user_total_orders = (
    prior_with_user
    .groupby("user_id")["order_id"]
    .count()
    .reset_index(name="user_total_orders")
)

user_total_orders["user_is_newbie"] = (user_total_orders["user_total_orders"] <= 5).astype(int)
user_total_orders["user_is_veteran"] = (user_total_orders["user_total_orders"] >= 50).astype(int)

print("Tamam: user_total_orders, user_is_newbie, user_is_veteran")


# 3) Zamanlama FE

# Hafta sonu oranı
weekend_mask = prior_with_user["order_dow"].isin([0, 6])
user_weekend_orders = (
    prior_with_user[weekend_mask]
    .groupby("user_id")["order_id"]
    .count()
)

user_weekend_ratio = (
    user_weekend_orders / user_total_orders.set_index("user_id")["user_total_orders"]
).fillna(0).reset_index(name="user_weekend_ratio")

# En sık sipariş günü
user_favorite_dow = (
    prior_with_user
    .groupby("user_id")["order_dow"]
    .agg(lambda x: x.mode()[0])
    .reset_index(name="user_favorite_dow")
)

# Peak saat (10–15)
peak_mask = prior_with_user["order_hour_of_day"].between(10, 15)
user_peak = (
    prior_with_user[peak_mask]
    .groupby("user_id")["order_id"]
    .count()
)

user_peak_ratio = (
    user_peak / user_total_orders.set_index("user_id")["user_total_orders"]
).fillna(0).reset_index(name="user_peak_hour_ratio")

# Gece kullanıcıları (20+)
night_mask = prior_with_user["order_hour_of_day"] >= 20
user_night = (
    prior_with_user[night_mask]
    .groupby("user_id")["order_id"]
    .count()
)

user_night_ratio = (
    user_night / user_total_orders.set_index("user_id")["user_total_orders"]
).fillna(0).reset_index(name="user_night_ratio")

print("Tamam: user_weekend_ratio, user_favorite_dow, user_peak_hour_ratio, user_night_ratio")


# 4) Sipariş sıklığı FE

user_avg_gap = (
    prior_with_user.groupby("user_id")["days_since_prior_order"]
    .mean()
    .reset_index(name="user_avg_days_between_orders")
)

user_max_gap = (
    prior_with_user.groupby("user_id")["days_since_prior_order"]
    .max()
    .reset_index(name="user_max_gap")
)

# Haftalık (7+-2 gün)
weekly_mask = prior_with_user["days_since_prior_order"].between(5, 9)
user_weekly_orders = (
    prior_with_user[weekly_mask]
    .groupby("user_id")["order_id"]
    .count()
)

user_weekly_ratio = (
    user_weekly_orders / user_total_orders.set_index("user_id")["user_total_orders"]
).fillna(0).reset_index(name="user_is_weekly_shopper")

# Aylık (30+-2 gün)
monthly_mask = prior_with_user["days_since_prior_order"].between(28, 32)
user_monthly_orders = (
    prior_with_user[monthly_mask]
    .groupby("user_id")["order_id"]
    .count()
)

user_monthly_ratio = (
    user_monthly_orders / user_total_orders.set_index("user_id")["user_total_orders"]
).fillna(0).reset_index(name="user_is_monthly_shopper")

print("Tamam: user_avg_days_between_orders, user_max_gap, weekly + monthly shopper")


# 5) Sadakat  + Keşif 

# Genel reorder oranı
user_reorder_ratio = (
    prior_with_user.groupby("user_id")["reordered"]
    .mean()
    .reset_index(name="user_reorder_ratio")
)

# Son 10 sipariş --> yeni ürün keşif oranı
last_10 = prior_with_user.sort_values(["user_id", "order_number"], ascending=[True, False])
last_10 = last_10.groupby("user_id").head(100)

user_exploration = (
    1 - last_10.groupby("user_id")["reordered"].mean()
).reset_index(name="user_exploration_score")

print("Tamam: user_reorder_ratio, user_exploration_score")


# 6) TÜM USER FEATURES BİRLEŞTİR

user_features = (
    user_total_orders
    .merge(user_weekend_ratio, on="user_id", how="left")
    .merge(user_favorite_dow, on="user_id", how="left")
    .merge(user_peak_ratio, on="user_id", how="left")
    .merge(user_night_ratio, on="user_id", how="left")
    .merge(user_avg_gap, on="user_id", how="left")
    .merge(user_max_gap, on="user_id", how="left")
    .merge(user_weekly_ratio, on="user_id", how="left")
    .merge(user_monthly_ratio, on="user_id", how="left")
    .merge(user_reorder_ratio, on="user_id", how="left")
    .merge(user_exploration, on="user_id", how="left")
)

print("\nUser features hazır:", user_features.shape)
print("Toplam User Feature Sayısı:", len(user_features.columns) - 1)


Temiz başlangıç yapıldı.

prior_with_user: (32434489, 9)
Tamam: user_total_orders, user_is_newbie, user_is_veteran
Tamam: user_weekend_ratio, user_favorite_dow, user_peak_hour_ratio, user_night_ratio
Tamam: user_avg_days_between_orders, user_max_gap, weekly + monthly shopper
Tamam: user_reorder_ratio, user_exploration_score

User features hazır: (206209, 14)
Toplam User Feature Sayısı: 13


hata çözüldü:  blok 2-3. kez çalıştığında merge üst üste binmesi, diğer tablolarda ürettiğim FE'ler için kod bloğunun başına ekleniyor.

vars_to_clear bloğu sayesinde FE'leri blok çalışmadan temizliyor ve üst üste binmesini engelliyor. 

Products tablosundan çıkaracağım FE’ler:

- product_order_count : Ürünün toplam kaç kez sipariş edildiği
- product_reorder_rate : Ürünün tekrar satın alınma oranı
- product_unique_users : Bu ürünü satın alan farklı kullanıcı sayısı
- product_avg_cart_position : Ürünün sepetteki ortalama eklenme sırası
- product_organic_flag : Ürün isminin “Organic” içerip içermediğini gösteren ikili sinyal
- product_department_popularity : Ürünün bulunduğu departmanın sipariş yoğunluğu
- product_aisle_popularity : Ürünün bulunduğu koridorun sipariş yoğunluğu

In [3]:
vars_to_clear = [
    "product_features",
    "products_with_prior",
    "product_order_count",
    "product_reorder_rate",
    "product_unique_users",
    "product_avg_cart_position",
    "product_organic_flag",
    "product_aisle_popularity",
]
for v in vars_to_clear:
    if v in globals():
        del globals()[v]

print("Product FE: temiz başlangıç yapıldı.\n")


# Prior + Products merge
products_with_prior = (
    order_products_prior
    .merge(products[['product_id', 'product_name', 'aisle_id', 'department_id']],
           on='product_id', how='left')
)

print("products_with_prior:", products_with_prior.shape)


# 1) product_order_count
product_order_count = (
    products_with_prior.groupby("product_id")["order_id"]
    .count().reset_index(name="product_order_count")
)


# 2) product_reorder_rate
product_reorder_rate = (
    products_with_prior.groupby("product_id")["reordered"]
    .mean().reset_index(name="product_reorder_rate")
)


# 3) product_unique_users
product_unique_users = (
    products_with_prior
    .merge(orders[['order_id', 'user_id']], on='order_id', how='left')
    .groupby("product_id")["user_id"]
    .nunique().reset_index(name="product_unique_users")
)


# 4) product_avg_cart_position
product_avg_cart_position = (
    products_with_prior.groupby("product_id")["add_to_cart_order"]
    .mean().reset_index(name="product_avg_cart_position")
)


# 5) product_organic_flag
products["product_organic_flag"] = (
    products["product_name"].str.contains("Organic", case=False, na=False).astype(int)
)

product_organic_flag = products[["product_id", "product_organic_flag"]]


# 6) aisle_popularity
aisle_popularity = (
    products_with_prior.groupby("aisle_id")["order_id"]
    .count().reset_index(name="aisle_total_orders")
)

product_aisle_popularity = (
    products[["product_id", "aisle_id"]]
    .merge(aisle_popularity, on="aisle_id", how="left")
)


# 7) TÜM PRODUCT FEATURES BİRLEŞTİR
product_features = (
    product_order_count
    .merge(product_reorder_rate, on="product_id", how="left")
    .merge(product_unique_users, on="product_id", how="left")
    .merge(product_avg_cart_position, on="product_id", how="left")
    .merge(product_organic_flag, on="product_id", how="left")
    .merge(product_aisle_popularity[["product_id", "aisle_total_orders"]], on="product_id", how="left")
)

print("\nProduct features hazır:", product_features.shape)
print("Toplam Product Feature Sayısı:", len(product_features.columns) - 1)


Product FE: temiz başlangıç yapıldı.

products_with_prior: (32434489, 7)

Product features hazır: (49677, 7)
Toplam Product Feature Sayısı: 6


Aisles tablosundan çıkaracağım FE’ler:
* aisle_total_orders : Bu koridordan geçen toplam sipariş sayısı
* aisle_unique_products : Koridorda bulunan farklı ürün sayısı
* aisle_unique_users : Bu koridordan alışveriş yapan farklı kullanıcı sayısı
* aisle_reorder_rate : Koridor içindeki ürünlerin ortalama yeniden satın alınma oranı
* aisle_popularity_rank : Aisle’ın genel kullanım sıralaması (1 = en popüler)

In [4]:
vars_to_clear = [
    "aisle_features", "aisle_total_orders",
    "aisle_unique_products", "aisle_unique_users",
    "aisle_reorder_rate", "aisle_popularity_rank",
    "products_with_prior_aisle"
]
for v in vars_to_clear:
    if v in globals():
        del globals()[v]

print("Aisle FE: temiz başlangıç yapıldı.\n")


# Prior + Aisle merge
products_with_prior_aisle = (
    order_products_prior
    .merge(products[['product_id', 'aisle_id']], on='product_id', how='left')
)

print("products_with_prior_aisle:", products_with_prior_aisle.shape)


# 1) aisle_total_orders
aisle_total_orders = (
    products_with_prior_aisle.groupby("aisle_id")["order_id"]
    .count().reset_index(name="aisle_total_orders")
)


# 2) aisle_unique_products
aisle_unique_products = (
    products_with_prior_aisle.groupby("aisle_id")["product_id"]
    .nunique().reset_index(name="aisle_unique_products")
)


# 3) aisle_unique_users
aisle_unique_users = (
    products_with_prior_aisle
    .merge(orders[['order_id', 'user_id']], on='order_id', how='left')
    .groupby("aisle_id")["user_id"]
    .nunique().reset_index(name="aisle_unique_users")
)


# 4) aisle_reorder_rate
aisle_reorder_rate = (
    products_with_prior_aisle.groupby("aisle_id")["reordered"]
    .mean().reset_index(name="aisle_reorder_rate")
)


# 5) aisle_popularity_rank
aisle_popularity_rank = (
    aisle_total_orders.sort_values("aisle_total_orders", ascending=False)
    .reset_index(drop=True)
)
aisle_popularity_rank["aisle_popularity_rank"] = aisle_popularity_rank.index + 1
aisle_popularity_rank = aisle_popularity_rank[["aisle_id", "aisle_popularity_rank"]]


# 6) BİRLEŞTİR
aisle_features = (
    aisle_total_orders
    .merge(aisle_unique_products, on="aisle_id", how="left")
    .merge(aisle_unique_users, on="aisle_id", how="left")
    .merge(aisle_reorder_rate, on="aisle_id", how="left")
    .merge(aisle_popularity_rank, on="aisle_id", how="left")
)

print("\nAisle features hazır:", aisle_features.shape)
print("Toplam Aisle Feature Sayısı:", len(aisle_features.columns) - 1)


Aisle FE: temiz başlangıç yapıldı.

products_with_prior_aisle: (32434489, 5)

Aisle features hazır: (134, 6)
Toplam Aisle Feature Sayısı: 5


> `Departments` tablosundan çıkaracağım FE’ler:

- department_total_orders : Bu departmandan toplam kaç ürün geçmiş (departmanın genel hacmi)
- department_unique_products : Departmandaki farklı ürün sayısı (çeşitlilik)
- department_unique_users : Bu departmandan alışveriş yapan kullanıcı sayısı
- department_reorder_rate : Departmandaki ürünlerin ortalama tekrar alma oranı
- department_popularity_rank : Departmanın kullanım yoğunluğuna göre sırası


In [5]:
vars_to_clear = [
    "department_features", "dept_total_orders",
    "dept_unique_products", "dept_unique_users",
    "dept_reorder_rate", "dept_popularity_rank",
    "products_with_prior_dept"
]
for v in vars_to_clear:
    if v in globals():
        del globals()[v]

print("Department FE: temiz başlangıç yapıldı.\n")


# Prior + Dept merge
products_with_prior_dept = (
    order_products_prior
    .merge(products[['product_id', 'department_id']], on='product_id', how='left')
)

print("products_with_prior_dept:", products_with_prior_dept.shape)


# 1) dept_total_orders
dept_total_orders = (
    products_with_prior_dept.groupby("department_id")["order_id"]
    .count().reset_index(name="department_total_orders")
)


# 2) dept_unique_products
dept_unique_products = (
    products_with_prior_dept.groupby("department_id")["product_id"]
    .nunique().reset_index(name="department_unique_products")
)


# 3) dept_unique_users
dept_unique_users = (
    products_with_prior_dept
    .merge(orders[['order_id', 'user_id']], on='order_id', how='left')
    .groupby("department_id")["user_id"]
    .nunique().reset_index(name="department_unique_users")
)


# 4) dept_reorder_rate
dept_reorder_rate = (
    products_with_prior_dept.groupby("department_id")["reordered"]
    .mean().reset_index(name="department_reorder_rate")
)


# 5) dept_popularity_rank
dept_popularity_rank = (
    dept_total_orders.sort_values("department_total_orders", ascending=False)
    .reset_index(drop=True)
)
dept_popularity_rank["department_popularity_rank"] = dept_popularity_rank.index + 1
dept_popularity_rank = dept_popularity_rank[["department_id", "department_popularity_rank"]]


# 6) BİRLEŞTİR
department_features = (
    dept_total_orders
    .merge(dept_unique_products, on="department_id", how="left")
    .merge(dept_unique_users, on="department_id", how="left")
    .merge(dept_reorder_rate, on="department_id", how="left")
    .merge(dept_popularity_rank, on="department_id", how="left")
)

print("\nDepartment features hazır:", department_features.shape)
print("Toplam Department Feature Sayısı:", len(department_features.columns) - 1)


Department FE: temiz başlangıç yapıldı.

products_with_prior_dept: (32434489, 5)

Department features hazır: (21, 6)
Toplam Department Feature Sayısı: 5


> `Order_Products_Prior` tablosundan çıkaracağım FE’ler:

- up_orders : Kullanıcı bu ürünü kaç defa almış
- up_last_order_number : Bu ürünü en son kaçıncı siparişinde almış
- up_order_rate : Kullanıcının toplam siparişlerine göre ürün alma oranı
- up_orders_since_last : Son siparişinden bu yana bu ürünü almayalı kaç sipariş geçti
- up_cart_mean : Bu ürün sepete ortalama kaçıncı sırada eklenmiş (user-product özel)
- up_reorder_rate : Kullanıcının bu ürünü tekrar alma ihtimali


In [6]:
vars_to_clear = [
    "up_features", "prior_with_user",
    "up_orders", "up_last_order_number",
    "up_order_rate", "up_orders_since_last",
    "up_reorder_rate", "up_cart_mean"
]
for v in vars_to_clear:
    if v in globals():
        del globals()[v]

print("User-Product FE: temiz başlangıç yapıldı.\n")


# Prior + user merge
prior_with_user = (
    order_products_prior
    .merge(
        orders[['order_id', 'user_id', 'order_number']],
        on='order_id', how='left'
    )
)


# Geçmişte kaç kez aldı
up_orders = (
    prior_with_user.groupby(["user_id", "product_id"])["order_id"]
    .count().reset_index(name="up_orders")
)


# En son hangi siparişte aldı
up_last_order_number = (
    prior_with_user.groupby(["user_id", "product_id"])["order_number"]
    .max().reset_index(name="up_last_order_number")
)


# User’ın toplam sipariş sayısı
user_total_orders_small = (
    orders.groupby("user_id")["order_id"]
    .count().reset_index(name="user_total_orders")
)

up_order_rate = (
    up_orders.merge(user_total_orders_small, on="user_id", how="left")
)
up_order_rate["up_order_rate"] = (
    up_order_rate["up_orders"] / up_order_rate["user_total_orders"]
)
up_order_rate = up_order_rate[["user_id", "product_id", "up_order_rate"]]


#up_reorder_rate
up_reorder_rate = (
    prior_with_user.groupby(["user_id", "product_id"])["reordered"]
    .mean().reset_index(name="up_reorder_rate")
)


#up_cart_mean
up_cart_mean = (
    prior_with_user.groupby(["user_id", "product_id"])["add_to_cart_order"]
    .mean().reset_index(name="up_cart_mean")
)


# En son alımından bu yana kaç sipariş geçti
train_max_order = (
    orders[orders['eval_set'] == 'train']
    .groupby("user_id")["order_number"]
    .max().reset_index(name="user_max_order_number")
)

up_orders_since_last = (
    up_last_order_number.merge(train_max_order, on="user_id", how="left")
)
up_orders_since_last["up_orders_since_last"] = (
    up_orders_since_last["user_max_order_number"] - up_orders_since_last["up_last_order_number"]
)
up_orders_since_last = up_orders_since_last[["user_id", "product_id", "up_orders_since_last"]]


#TÜM USER-PRODUCT FE BİRLEŞTİR
up_features = (
    up_orders
    .merge(up_last_order_number, on=["user_id", "product_id"], how="left")
    .merge(up_order_rate, on=["user_id", "product_id"], how="left")
    .merge(up_reorder_rate, on=["user_id", "product_id"], how="left")
    .merge(up_cart_mean, on=["user_id", "product_id"], how="left")
    .merge(up_orders_since_last, on=["user_id", "product_id"], how="left")
)

print("\nUser-Product features hazır:", up_features.shape)
print("Toplam User-Product Feature Sayısı:", len(up_features.columns) - 2)


User-Product FE: temiz başlangıç yapıldı.


User-Product features hazır: (13307953, 8)
Toplam User-Product Feature Sayısı: 6


> `Order_Products_Train` tablosundan çıkaracağım FE’ler:

- reordered (hedef değişken): Ürün tekrar alındı mı?
- train_user_id : Siparişi hangi kullanıcıya ait (orders tablosuyla merge’den gelir)
- train_full_merge : user_features + product_features + up_features birleştirilmiş final tablo
- final_feature_count : Üretilen toplam feature sayısı (kontrol amaçlı)



In [7]:
# Train orders
train_orders_full = orders[orders['eval_set'] == 'train'][['order_id', 'user_id']]

train_full = (
    order_products_train
    .merge(train_orders_full, on='order_id', how='left')
)


# User Features
train_full = train_full.merge(user_features, on="user_id", how="left")

# Product Features
train_full = train_full.merge(product_features, on="product_id", how="left")

# Aisle / Department IDs
train_full = train_full.merge(
    products[['product_id', 'aisle_id', 'department_id']],
    on='product_id', how='left'
)

# Aisle Features
train_full = train_full.merge(aisle_features, on="aisle_id", how="left")

# Department Features
train_full = train_full.merge(department_features, on="department_id", how="left")

# User–Product Interaction Features
train_full = train_full.merge(up_features, on=["user_id", "product_id"], how="left")

# Missing fill
numeric_cols = train_full.select_dtypes(include=np.number).columns
train_full[numeric_cols] = train_full[numeric_cols].fillna(0)

print("\nFinal training dataset hazır:", train_full.shape)
print("Toplam Feature Sayısı:", len(train_full.columns) - 5)



Final training dataset hazır: (1384617, 42)
Toplam Feature Sayısı: 37


In [8]:
train_full.to_csv(DATA_INTERIM / "train_features.csv", index=False)
user_features.to_csv(DATA_INTERIM / "user_features.csv", index=False)
product_features.to_csv(DATA_INTERIM / "product_features.csv", index=False)
aisle_features.to_csv(DATA_INTERIM / "aisle_features.csv", index=False)
department_features.to_csv(DATA_INTERIM / "department_features.csv", index=False)
up_features.to_csv(DATA_INTERIM / "up_features.csv", index=False)

print("Tüm feature dosyaları kaydedildi.")


Tüm feature dosyaları kaydedildi.


### FE:  

Aşağıdaki tablo, bu aşamada oluşturulan tüm feature setinin özetini gösterir:

| Kategori               | Feature Sayısı |
|------------------------|----------------|
| User Features          | 13             |
| Product Features       | 9              |
| Category Features      | 5              |
| User–Product Interaction | 10          |
| **Toplam**             | **37**         |

### Feature Engineering Özeti

EDA ve Baseline gözlemlenen analizler doğrultusunda: 

Bu aşamada Model için kullanıcı, ürün, kategori ve kullanıcı–ürün etkileşimlerinden oluşan toplam **37 feature** üretildi.  
Tüm sinyaller EDA sürecinde gözlenen davranış örüntüleriyle tutarlı şekilde tasarlandı.

Detaylı açıklamalar ve çıkarımlar **feature_engineering_report.md** dosyasında dokümante edilmiştir.
